In [1]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [11]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_swta_t')),
}

In [12]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [13]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,100,67.77,51.25
1,H-UNet-SWTA-T,0,5,100,58.24,41.08
2,H-UNet-SWTA-T,0,10,100,53.94,36.93
3,H-UNet-SWTA-T,0,20,100,63.43,46.44
4,H-UNet-SWTA-T,0,50,100,66.33,49.62
5,H-UNet-SWTA-T,0,100,100,64.64,47.75


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                Mean CI 0.9%    Mean CI 0.9%
Model         Inv Temp Regime                               
H-UNet-SWTA-T 1        100     67.77     NaN   51.25     NaN
              5        100     58.24     NaN   41.08     NaN
              10       100     53.94     NaN   36.93     NaN
              20       100     63.43     NaN   46.44     NaN
              50       100     66.33     NaN   49.62     NaN
              100      100     64.64     NaN   47.75     NaN

In [41]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_unet_swta_t')),
}

In [42]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [43]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,1,70.00,53.85
1,H-UNet-SWTA-T,0,1,2,68.69,52.31
2,H-UNet-SWTA-T,0,1,5,73.18,57.71
3,H-UNet-SWTA-T,0,1,10,79.81,66.39
4,H-UNet-SWTA-T,0,1,20,82.54,70.28
...,...,...,...,...,...,...
145,H-UNet-SWTA-T,4,100,1,68.61,52.22
146,H-UNet-SWTA-T,4,100,2,69.31,53.03
147,H-UNet-SWTA-T,4,100,5,78.49,64.60
148,H-UNet-SWTA-T,4,100,10,79.25,65.63


Dice           Jaccard          
                                 Mean   CI 0.9%    Mean   CI 0.9%
Model         Inv Temp Regime                                    
H-UNet-SWTA-T 1        1       68.466  0.746855  52.058  0.872077
                       2       68.316  0.238126  51.878  0.274696
                       5       71.600  2.307726  55.818  2.786038
                       10      78.968  0.985244  65.252  1.350338
                       20      78.468  5.354409  64.898  6.895587
              5        1       68.102  0.262375  51.636  0.304823
                       2       68.208  0.713463  51.756  0.829255
                       5       75.858  2.701819  61.182  3.439453
                       10      79.154  1.108898  65.514  1.517463
                       20      82.464  0.823182  70.168  1.193779
              10       1       68.262  0.373428  51.816  0.431733
                       2       69.036  0.635064  52.718  0.745096
                       5       74.690  2.699285  59.682  3.432257
                       10      80.676  1.301378  67.630  1.809907
                       20      82.798  1.367834  70.668  1.998926
              20       1       68.160  0.379349  51.700  0.439305
                       2       68.276  0.396574  51.834  0.456817
                       5       75.236  2.142629  60.352  2.726247
                       10      79.484  1.279742  65.974  1.759337
                       20      83.270  1.750183  71.374  2.559457
              50       1       67.840  0.093413  51.332  0.108603
                       2       69.750  1.093984  53.560  1.280486
                       5       74.950  2.699480  60.020  3.405258
                       10      78.994  0.681460  65.286  0.931216
                       20      83.186  1.723372  71.250  2.528439
              100      1       68.014  0.305657  51.532  0.352904
                       2       68.866  0.745856  52.520  0.866904
                       5       74.652  2.607704  59.630  3.315021
                       10      79.248  1.316458  65.648  1.808413
                       20      82.016  1.163584  69.530  1.659043

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [44]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'UNet': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('unet')),
    #'EM': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('em_unet')),
    #'UAMT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('uamt_unet')),
}

In [45]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [46]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,UNet,0,1,100,90.38,82.44
1,UNet,1,1,100,91.26,83.92
2,UNet,2,1,100,90.71,82.99
3,UNet,3,1,100,90.61,82.83
4,UNet,4,1,100,90.76,83.09
5,UNet,5,1,100,90.17,82.10
6,UNet,6,1,100,91.14,83.71
7,UNet,7,1,100,90.26,82.24
8,UNet,8,1,100,90.77,83.09
9,UNet,9,1,100,90.21,82.17


Dice           Jaccard          
                         Mean   CI 0.9%    Mean   CI 0.9%
Model Inv Temp Regime                                    
UNet  1        100     90.627  0.208009  82.858  0.347043